In [8]:
search_for = 91
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00431060791015625
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 86665741
type: [1, 1, 1, 1, 91] 91
cases of this type: 68574961
10000 11.905069631260734
20000 11.309394564304776
30000 27.722563802489045
40000 41.81263109473709
50000 37.28711716061427
60000 42.75081325693749
70000 45.425366832173644
80000 47.41483097125588
90000 53.910304748442165
100000 49.33370939925812
110000 52.24286227109922
120000 49.77732799678056
130000 47.45103481598016
140000 49.20823146057926
150000 49.557283933986206
160000 49.32100169780332
170000 48.09026814506271
180000 50.961655439150825
190000 55.36173952225877
200000 54.846567522712725
210000 48.83954040951512
220000 48.479011665202144
230000 49.798587485350325
240000 49.798535012141244
250000 46.94743603163087
260000 46.72761259618094
270000 47.49179792390787
280000 49.512544029455874
290000 47.20843320212767
300000 49.349235120304755
310000 48.4857236996866

3090000 45.132602839644655
3100000 45.255079657286984
3110000 43.26300898052514
3120000 45.2682386945742
3130000 42.85614363364405
3140000 40.7265999710312
3150000 42.69771621751611
3160000 42.717408483075225
3170000 42.65038406565408
3180000 44.92335026312475
3190000 47.38438356281875
3200000 44.55174059460779
3210000 45.020830373265504
3220000 45.71941150862234
3230000 45.89560754639154
3240000 45.263187254760034
3250000 42.47610271756825
3260000 42.511778613077084
3270000 42.206443041723205
3280000 40.72706230025971
3290000 42.361484562143076
3300000 42.19603363123782
3310000 43.79734950634497
3320000 46.633592960069855
3330000 44.17972971243764
3340000 45.14261477537163
3350000 38.87827989071356
3360000 42.40152178137907
3370000 49.13668362923736
3380000 8.087722756513072
3390000 7.27996968785674
3400000 7.273285990941464
3410000 7.208181565757327
3420000 45.32919036393208
3430000 41.830865214723865
3440000 41.88711401085064
3450000 39.22686001981357
3460000 46.345931779009696
3470

6210000 41.504126174053475
6220000 43.23003778367573
6230000 41.67040590669744
6240000 41.04533461988037
6250000 43.661512564497016
6260000 45.42546001688082
6270000 46.714260400051295
6280000 42.1696225920133
6290000 43.70879715973544
6300000 42.150073093453024
6310000 40.87508739415711
6320000 40.66944509004733
6330000 42.12577605629094
6340000 39.340801575954096
6350000 42.56803351105372
6360000 39.39907855698873
6370000 36.93717820483071
6380000 45.286370070177256
6390000 24.862484650468694
6400000 7.205936051663557
6410000 7.219717079981943
6420000 7.208380412460817
6430000 25.533411340871158
6440000 45.09214795481734
6450000 39.724917292015135
6460000 37.233490526596256
6470000 41.08815004588498
6480000 40.1684589779987
6490000 40.908132432221116
6500000 40.204616075800146
6510000 38.87436182118918
6520000 40.893353107799186
6530000 40.81796243651923
6540000 42.25372117291938
6550000 44.71846638146973
6560000 44.7480326172288
6570000 44.931489914403734
6580000 42.12794182089275
6

9330000 36.4611246968756
9340000 36.43649313299376
9350000 37.05249414332011
9360000 36.54409894042056
9370000 35.964074557318924
9380000 35.38643783264865
9390000 38.41563696285522
9400000 42.597874293650364
9410000 6.644338991932339
9420000 6.66055031604906
9430000 6.7854110860438945
9440000 6.860501189199381
9450000 45.79318664112288
9460000 40.476760549843625
9470000 33.5534670618837
9480000 38.8860481181574
9490000 36.45549215302624
9500000 36.67024283629494
9510000 35.640235917809655
9520000 35.559874886409645
9530000 39.0200436049021
9540000 39.25025088881698
9550000 40.75640051480571
9560000 42.08714528681322
9570000 41.99579530358132
9580000 38.79921316563522
9590000 41.164802819965374
9600000 39.12919250714703
9610000 40.401168494236344
9620000 42.15353943375409
9630000 43.92050771511359
9640000 45.92400596115069
9650000 40.600514356074946
9660000 42.02475827441735
9670000 46.60490777285188
9680000 23.11933466837204
9690000 16.643573495154573
9700000 17.25319373297024
9710000

12370000 35.06945037252834
12380000 34.92825428290056
12390000 33.94619603562598
12400000 33.375570524335565
12410000 35.10814802007011
12420000 18.28860743516565
12430000 6.405427597010612
12440000 6.405232801494936
12450000 6.388260083275868
12460000 24.96405061778599
12470000 38.50464969003341
12480000 33.71418887959326
12490000 33.649296153618096
12500000 34.9784458596744
12510000 32.72202170364824
12520000 33.56144239835437
12530000 34.52931881288682
12540000 35.326722714912364
12550000 33.89904952608738
12560000 35.84182828436891
12570000 37.92401327556429
12580000 37.550980772178946
12590000 35.927068551366126
12600000 34.956486627968744
12610000 35.61641154657719
12620000 36.97171004510883
12630000 38.88237352467631
12640000 36.49115931791732
12650000 38.099828938250866
12660000 24.8867737173158
12670000 15.518486452304543
12680000 15.340935733718515
12690000 15.5911103210983
12700000 33.355900666382134
12710000 47.15925698577087
12720000 38.28942349809576
12730000 37.643453289

...

75040000 1.1422085536875126
75050000 1.2553996278754076
75060000 1.8222915539319051
75070000 1.8363253258991836
75080000 1.8900043945485883
75090000 1.523945583734082
75100000 1.2397783450629711
75110000 1.3632070761913986
75120000 1.7305683966162138
75130000 2.7185842707362773
75140000 2.9681330211654973
75150000 2.1665210418072474
75160000 1.784583876964291
75170000 2.181659635422051
75180000 2.767857857340289
75190000 3.1328885630110106
75200000 2.1854099236118727
75210000 1.8766754145096274
75220000 1.7315021376824975
75230000 2.1479543795440263
75240000 2.1021471927720308
75250000 2.5368849361901877
75260000 2.5311458798298836
75270000 1.89764580429048
75280000 1.8545031993852255
75290000 2.072773033342355
75300000 2.2503086335604916
75310000 2.7936337270068528
75320000 2.846339587013377
75330000 1.9733741518550711
75340000 2.26819396624893
75350000 2.9610826499875653
75360000 2.1546686783845623
75370000 2.357730045977791
75380000 2.9955565954501897
75390000 2.071733334702227
7540

78000000 0.8286601332902114
78010000 1.1061310783841014
78020000 1.2638478800516393
78030000 1.3465766064382791
78040000 1.2528539410588742
78050000 1.0015287519766622
78060000 0.8504253460742764
78070000 1.2127557548340957
78080000 1.2495191537314785
78090000 1.6038819133583637
78100000 1.4733389722594021
78110000 0.8403508645911283
78120000 0.8287209869225488
78130000 1.2702622614694634
78140000 1.6124881459927227
78150000 2.62218974449467
78160000 1.8890554471021297
78170000 1.410669717293951
78180000 1.5510331890105142
78190000 1.9115809786928297
78200000 2.5340908126542305
78210000 2.0254026932933726
78220000 1.7821863652211227
78230000 1.2473281386335757
78240000 1.2563723908077213
78250000 1.6465423019752699
78260000 1.9342494279743896
78270000 1.971380991340412
78280000 1.4420661388447882
78290000 1.104298551952243
78300000 1.5793883840174079
78310000 1.741850993162692
78320000 1.7940123597993318
78330000 2.057902300812873
78340000 1.8083182015116215
78350000 1.565327810476972


80930000 0.8931511013540028
80940000 0.8885894873891607
80950000 1.1124246924407881
80960000 1.2109251059582697
80970000 1.1975209079542293
80980000 0.825035271210909
80990000 0.8449798773888547
81000000 0.46246482194561433
81010000 0.644939523456653
81020000 0.8358243812549379
81030000 0.6545995006095171
81040000 0.7784670011860927
81050000 1.0388113091989029
81060000 0.9955443304924964
81070000 1.35121395109421
81080000 1.1585478876198265
81090000 0.7925173867689372
81100000 0.7372266525786519
81110000 0.7450359435130688
81120000 0.6790402163236473
81130000 0.8125964869547287
81140000 0.6741748322518931
81150000 0.7261630329888066
81160000 1.0529559729953408
81170000 0.8647531197414597
81180000 1.1781408008070058
81190000 1.0658197366496525
81200000 0.8893430856412584
81210000 0.8878233110222552
81220000 0.9035364246914188
81230000 0.6853930690805514
81240000 0.8164696317844523
81250000 0.7566493286815881
81260000 0.7059839144946204
81270000 1.2278179990241196
81280000 1.078839315097

83840000 0.17871836428886653
83850000 0.17387174955794546
83860000 0.17836386173039676
83870000 0.17804078493111666
83880000 0.18520766851500006
83890000 0.1815039504688581
83900000 0.15116735028892092
83910000 0.17435845045155288
83920000 0.18177514076203108
83930000 0.1802101090120938
83940000 0.17348510743967693
83950000 0.14626720802825688
83960000 0.16918047223887842
83970000 0.1710778448923561
83980000 0.16097956349893408
83990000 0.17167985566151803
84000000 0.17741426996479434
84010000 0.1331735137073199
84020000 0.167151878948715
84030000 0.17139127133682702
84040000 0.15939473661196232
84050000 0.1711352150570353
84060000 0.1539540761054953
84070000 0.10282825623172522
84080000 0.15693783177906276
84090000 0.15590453704467747
84100000 0.17497955694570147
84110000 0.136668747084962
84120000 0.1716489793407652
84130000 0.16870623054236172
84140000 0.16223705513057443
84150000 0.16841930479909312
84160000 0.1560429773229162
84170000 0.16556675406854685
84180000 0.147322356762978

86550000 0.01537229614546564
86560000 0.012551554144084452
86570000 0.017714172578765286
best so far: 0
type: [1, 7, 1, 13, 1] 91
cases of this type: 15379
86580000 0.013899321685519484
86590000 0.005105626483500003
best so far: 0
type: [1, 7, 13, 1, 1] 91
cases of this type: 1183
best so far: 0
type: [1, 13, 1, 1, 7] 91
cases of this type: 31213
86600000 0.005751754693051179
86610000 0.004395117922948466
86620000 0.0035539738702178
best so far: 0
type: [1, 13, 1, 7, 1] 91
cases of this type: 4459
86630000 0.0032811774537629553
best so far: 0
type: [1, 13, 7, 1, 1] 91
cases of this type: 637
best so far: 0
type: [1, 91, 1, 1, 1] 91
cases of this type: 91
best so far: 0
type: [7, 1, 1, 1, 13] 91
cases of this type: 28561
86640000 0.0029076055282685493
86650000 0.0013261913394927978
86660000 0.0006704444525308079
best so far: 0
type: [7, 1, 1, 13, 1] 91
cases of this type: 2197
best so far: 0
type: [7, 1, 13, 1, 1] 91
cases of this type: 169
best so far: 0
type: [7, 13, 1, 1, 1] 91
cases